In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pymysql,calendar,time,json
import requests
from datetime import datetime
from threading import Timer


In [14]:
url = "http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13"
response = requests.get(url)   
html = response.text   
soup = BeautifulSoup(html,"html")



In [17]:
url = "http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13"
html = pd.read_html(url)
df = html[0]


In [19]:
df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,AJ네트웍스,95570,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,손삼달,http://www.ajnet.co.kr,서울특별시
1,BGF리테일,282330,종합 소매업,체인화 편의점,2017-12-08,12월,민승배,http://www.bgfretail.com,서울특별시
2,BNK금융지주,138930,기타 금융업,금융지주회사,2011-03-30,12월,빈대인,http://www.bnkfg.com,부산광역시
3,DSR,155660,1차 비철금속 제조업,합섬섬유로프,2013-05-15,12월,홍석빈,http://www.dsr.com,부산광역시
4,DS단석,17860,기타 화학제품 제조업,바이오에너지,2023-12-22,12월,한승욱,http://dsdansuk.com,경기도


In [49]:
url = "https://finance.naver.com/item/sise_day.nhn?code=005930"
response = requests.get(url,headers={"user-agent":"Mozilla/5.0"})
html = response.text


In [50]:

table = pd.read_html(html)[0]
stock_data = table.dropna()
stock_data

df = df.append(pd.read_html(requests.get(pg_url,headers={"user-agent":"Mozilla/5.0"}).text)[0])


,날짜,종가,전일비,시가,고가,저가,거래량
1,2024.01.10,74100.0,600.0,75000.0,75200.0,74000.0,6570752.0
2,2024.01.09,74700.0,1800.0,77400.0,77700.0,74300.0,26019249.0
3,2024.01.08,76500.0,100.0,77000.0,77500.0,76400.0,11088724.0
4,2024.01.05,76600.0,0.0,76700.0,77100.0,76400.0,11304316.0
5,2024.01.04,76600.0,400.0,76100.0,77300.0,76100.0,15324439.0
9,2024.01.03,77000.0,2600.0,78500.0,78800.0,77000.0,21753644.0
10,2024.01.02,79600.0,1100.0,78200.0,79800.0,78200.0,17142847.0
11,2023.12.28,78500.0,500.0,77700.0,78500.0,77500.0,17797536.0
12,2023.12.27,78000.0,1400.0,76700.0,78000.0,76500.0,20651042.0
13,2023.12.26,76600.0,700.0,76100.0,76700.0,75700.0,13164909.0


In [63]:
soup = BeautifulSoup(response.text,"lxml")
            
last_page = soup.find("td",class_="pgRR").a.attrs["href"]
last_page.split("page=")[1]


'691'

In [16]:
class DBUpdater: 
    def __init__(self):        
        self.conn = pymysql.connect(host='localhost', user='root', password='11111111', db='INVESTAR', charset='utf8')
            
        with self.conn.cursor() as curs:
            sql = """
            CREATE TABLE IF NOT EXISTS company_info (
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price(
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
            
        self.conn.commit()    
            
        self.codes = dict()
        self.update_comp_info()
    
    def __del__(self):
        self.conn.close() 
        
    def read_krx_code(self):
        url = 'http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
            
        krx = pd.read_html(url, header=0)[0]  
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드': 'code', '회사명': 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        
        return krx
    
    # 종목코드를 company_info 테이블에 업데이트 한 후 딕셔너리에(self.codes) 저장
    def update_comp_info(self):
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn)
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx]
            
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info"
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d')
            
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today:
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]   
                    sql = f"REPLACE INTO company_info (code, company, last"f"_update) VALUES ('{code}', '{company}', '{today}')"
                    curs.execute(sql)
                    self.codes[code] = company

                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] #{idx+1:04d} REPLACE INTO company_info"f"VALUES ({code}, {company}, {today})")    
                    
                self.conn.commit()
                
    def read_naver(self, code, company, pages_to_fetch):
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            html = BeautifulSoup(requests.get(url,headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
            pgrr = html.find("td", class_="pgRR")
            
            if pgrr is None:
                return None
            
            s = str(pgrr.a["href"]).split('=')
            lastpage = s[-1]
            
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url, page)
                tmpDF = pd.read_html(requests.get(pg_url, headers={"user-agent": "Mozilla/5.0"}).text)[0]
                df = pd.concat([df,tmpDF],ignore_index=True)
                
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.
                    format(tmnow, company, code, page, pages), end="\r")
                
            df = df.rename(columns={'날짜':'date','종가':'close','전일비':'diff'
                ,'시가':'open','고가':'high','저가':'low','거래량':'volume'})
            df['date'] = df['date'].replace('.', '-')
            df = df.dropna()
            
            df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close',
                'diff', 'open', 'high', 'low', 'volume']].astype(int)
            df = df[['date', 'open', 'high', 'low', 'close', 'diff', 'volume']]
            
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        
        return df
    
    # 看想看的公司的股价
    def update_daily_price(self,pages_to_fetch): 
        
        for idx,code in enumerate(self.codes):
            if code == "005930":
                df = self.read_naver(code,self.codes[code],pages_to_fetch)
                if df is None:
                    continue
                self.replace_into_db(df,idx,code, self.codes[code])
            
            #if idx < 2: #company list的前两个公司
                #return 
            
    def replace_into_db(self, df, num, code, company):
    # 네이버 금융에서 읽어온 주식 시세를 DB에 REPLACE
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = """
                    REPLACE INTO daily_price 
                    VALUES ('{code}', '{date}', {open}, {high}, {low}, {close}, {diff}, {volume})
                """.format(
                    code=code, 
                    date=r.date, 
                    open=r.open, 
                    high=r.high, 
                    low=r.low, 
                    close=r.close, 
                    diff=r.diff, 
                    volume=r.volume
                )
                curs.execute(sql)
        self.conn.commit()
        
        print('[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_price [OK]'.format(
            datetime.now().strftime('%Y-%m-%d %H:%M'), num + 1, company, code, len(df)
        ))


    def execute_daily(self):
        self.update_comp_info()
        try:
            with open("config.json", "r") as in_file:
                config = json.load(in_file)
                pages_to_fetch = config.get("pages_to_fetch", 100)  # 设置默认值为100，以防配置文件中没有 "pages_to_fetch" 键

        except FileNotFoundError:
            with open("config.json", "w") as out_file:
                pages_to_fetch = 100
                config = {"pages_to_fetch": pages_to_fetch}  # 修正为正确的键名
                json.dump(config, out_file)
   
        
        self.update_daily_price(pages_to_fetch)
        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year,tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year=tmnow.year+1,month=1,day=1,hour=17, minute=0,second=0)
        
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1,day=1,hour=17, minute=0,second=0)
            
        else:
            tmnext = tmnow.replace(day = tmnow.day+1,hour=17, minute=0,second=0)
            
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        
        t = Timer(secs,self.execute_daily)
        t.start()
                   

In [17]:
dbu = DBUpdater()
dbu.execute_daily()

/var/folders/qf/y4fxq3bj22d5xm0zz6d8kh9w0000gn/T/ipykernel_10466/1554565453.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


[2024-01-11 13:37] #0260 삼성전자 (005930) : 1000 rows > REPLACE INTO daily_price [OK]


/var/folders/qf/y4fxq3bj22d5xm0zz6d8kh9w0000gn/T/ipykernel_10466/1554565453.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


[2024-01-11 17:11] #0260 삼성전자 (005930) : 1000 rows > REPLACE INTO daily_price [OK]


/var/folders/qf/y4fxq3bj22d5xm0zz6d8kh9w0000gn/T/ipykernel_10466/1192849731.py:50: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, self.conn)


[2024-01-12 02:25] #0001 REPLACE INTO company_infoVALUES (095570, AJ네트웍스, 2024-01-12)
[2024-01-12 02:25] #0002 REPLACE INTO company_infoVALUES (282330, BGF리테일, 2024-01-12)
[2024-01-12 02:25] #0003 REPLACE INTO company_infoVALUES (138930, BNK금융지주, 2024-01-12)
[2024-01-12 02:25] #0004 REPLACE INTO company_infoVALUES (155660, DSR, 2024-01-12)
[2024-01-12 02:25] #0005 REPLACE INTO company_infoVALUES (017860, DS단석, 2024-01-12)
[2024-01-12 02:25] #0006 REPLACE INTO company_infoVALUES (009540, HD한국조선해양, 2024-01-12)
[2024-01-12 02:25] #0007 REPLACE INTO company_infoVALUES (322000, HD현대에너지솔루션, 2024-01-12)
[2024-01-12 02:25] #0008 REPLACE INTO company_infoVALUES (001390, KG케미칼, 2024-01-12)
[2024-01-12 02:25] #0009 REPLACE INTO company_infoVALUES (033180, KH 필룩스, 2024-01-12)
[2024-01-12 02:25] #0010 REPLACE INTO company_infoVALUES (025000, KPX케미칼, 2024-01-12)
[2024-01-12 02:25] #0011 REPLACE INTO company_infoVALUES (058860, KTis, 2024-01-12)
[2024-01-12 02:25] #0012 REPLACE INTO company_infoVALUE

In [26]:
import pandas as pd
import pymysql
from datetime import datetime
from datetime import timedelta

def fetch_data_from_db(code,start_time,end_time):
    
    if start_time is None and end_time is None:
        start_time=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
        end_time = datetime.now().strftime("%Y-%m-%d")   

    elif start_time is None and end_time is not None: 
        start_time = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
        
    elif end_time is None and start_time is not None:
        end_time = datetime.now().strftime("%Y-%m-%d")    

    conn = pymysql.connect(host='localhost', user='root', password='11111111', db='INVESTAR', charset='utf8')
    sql = f"SELECT * FROM daily_price WHERE date BETWEEN '{start_time}' AND '{end_time}' AND code = '{code}'"

    df = pd.read_sql(sql=sql,con=conn)
    return df

data = fetch_data_from_db(start_time=None,end_time=None,code="005930")
data

/var/folders/qf/y4fxq3bj22d5xm0zz6d8kh9w0000gn/T/ipykernel_10466/3459895060.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sql,con=conn)


,code,date,open,high,low,close,diff,volume
0,005930,2023-01-12,61100,61200,59900,60500,0,16102561
1,005930,2023-01-13,60500,61200,60400,60800,300,12510328
2,005930,2023-01-16,61300,61600,60800,61100,300,10039972
3,005930,2023-01-17,61200,61500,60600,61000,100,9831456
4,005930,2023-01-18,60700,61000,59900,60400,600,11584041
...,...,...,...,...,...,...,...,...
240,005930,2024-01-05,76700,77100,76400,76600,0,11304316
241,005930,2024-01-08,77000,77500,76400,76500,100,11088724
242,005930,2024-01-09,77400,77700,74300,74700,1800,26019249
243,005930,2024-01-10,75000,75200,73200,73600,1100,20259529


In [32]:
import pandas as pd
import pymysql
from datetime import datetime
from datetime import timedelta
import re


class MarketDB():
    def __init__(self):
        self.conn = pymysql.connect(host='localhost', user='root', password='11111111', db='INVESTAR', charset='utf8')
 
        
    
    def __del__(self):
        self.conn.close() 
    
    def get_daily_price(self,code,start_time = None, end_time = None):
        # 使用函数时如果没有填入日期 default取数期间设置为一年
        # 1 如果没有填入start time, 取数范围为1年前～end_time, 2 如果没有填入end time取数区间为 start_time~现在时间
        if start_time is None and end_time is None:
            start_time=(datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
            end_time = datetime.now().strftime("%Y-%m-%d")   

        elif start_time is None and end_time is not None: 
            start_time = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")
        
        elif end_time is None and start_time is not None:
            end_time = datetime.now().strftime("%Y-%m-%d") 
            
     
        
        sql = f"SELECT * FROM daily_price WHERE date BETWEEN '{start_time}' AND '{end_time}' AND code = '{code}'"
        df = pd.read_sql(sql=sql,con=self.conn)
        
        return df 
    

mk = MarketDB()
raw_df =mk.get_daily_price("000020","","2023-12-10")
raw_df

/var/folders/qf/y4fxq3bj22d5xm0zz6d8kh9w0000gn/T/ipykernel_10466/1101526496.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=sql,con=self.conn)


,code,date,open,high,low,close,diff,volume
0,000020,2019-12-23,8120,8240,8000,8150,10,115949
1,000020,2019-12-24,8140,8180,7990,8150,0,150877
2,000020,2019-12-26,8160,8160,8000,8000,150,318154
3,000020,2019-12-27,8010,8150,7980,8140,140,57641
4,000020,2019-12-30,8110,8420,8110,8310,170,198584
...,...,...,...,...,...,...,...,...
974,000020,2023-12-04,9450,9520,9300,9350,80,71430
975,000020,2023-12-05,9350,9550,9330,9550,200,78661
976,000020,2023-12-06,9600,9690,9510,9600,50,62958
977,000020,2023-12-07,9620,10480,9600,9910,310,1865532
